In [1]:
import pandas as pd
import numpy as np
import pickle
import os, sys
import re
import glob

#
import pathmagic
import initialise as init

In [2]:
pd.set_option('display.max_columns', 100)

### Accident data

In [3]:
# load data
# all_files = glob.glob(init.raw_path + "/road_safety_data/dft-road-casualty-statistics-accident-2021.csv")

# acc_files = [pd.read_csv(file) for file in all_files] 

# df_acc = pd.concat(acc_files)

# this file has lsoa
df_acc = pd.read_csv(init.raw_path + "/road_safety_data/dft-road-casualty-statistics-accident-1979-2021.csv")

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,2,15,16,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_acc['accident_year'].value_counts()

1989    260759
1990    258441
1982    256007
1979    254967
1984    253183
1980    250958
1981    248276
1986    247878
1988    246994
1985    245645
1983    242876
1997    240287
1987    239063
1998    238923
1996    236193
1991    235889
1999    235048
1994    234254
2000    233729
1992    233104
1995    230544
2001    229014
1993    228975
2002    221751
2003    214030
2004    207410
2005    198735
2006    189161
2007    182115
2008    170591
2009    163554
2010    154414
2011    151474
2014    146322
2012    145571
2015    140056
2013    138660
2016    136621
2017    129982
2018    122635
2019    117536
2021    101087
2020     91199
Name: accident_year, dtype: int64

In [5]:
# check if missings (and if all in same year - file)
df_acc[df_acc.isna().any(axis =1)].groupby('accident_year')['accident_index'].count()

accident_year
1979    254967
1980    250958
1981    248276
1982    256007
1983    242876
1984    253183
1985    245645
1986    247878
1987    239063
1988    246994
1989    260759
1990    258441
1991    235889
1992    233104
1993    228975
1994    234254
1995    230544
1996    236193
1997    240287
1998    238923
1999      1368
2000       881
2001       801
2002       465
2003       270
2004       130
2005        69
2006        17
2007        15
2008        10
2015        27
2016        44
2017        29
2018        55
2019        28
2020        14
2021        17
Name: accident_index, dtype: int64

* 8 missings area high for <= 1998
* decide range to use (probably >2012)

In [9]:
df_acc.tail()

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
8703906,2021991196247,2021,991196247,325546.0,674548.0,-3.194076,55.958152,99,3,2,1,22/09/2021,4,09:05,-1,S12000036,S12000036,6,0,6,20.0,3,4,6,0,0,0,1,1,1,0,0,1,1,-1,-1
8703907,2021991196607,2021,991196607,271195.0,558271.0,-4.010304,54.902512,99,3,2,2,11/09/2021,7,21:15,-1,S12000006,S12000006,3,711,6,60.0,0,-1,0,-1,0,0,6,1,1,0,0,2,1,-1,-1
8703908,2021991197944,2021,991197944,357297.0,860768.0,-2.716825,57.634805,99,3,1,1,17/04/2021,7,14:30,-1,S12000034,S12000034,4,9022,6,60.0,9,3,6,0,0,0,1,1,1,0,0,2,1,-1,-1
8703909,2021991200639,2021,991200639,326936.0,675925.0,-3.172192,55.970735,99,3,1,1,17/09/2021,6,17:00,-1,S12000036,S12000036,3,901,9,20.0,3,2,3,901,0,5,1,9,1,0,0,1,2,-1,-1
8703910,2021991201030,2021,991201030,270574.0,556367.0,-4.019126,54.885252,99,3,2,1,15/10/2021,6,16:15,-1,S12000006,S12000006,3,711,6,60.0,0,-1,0,-1,0,0,1,1,1,4,0,2,1,-1,-1


In [8]:
df_acc.shape

(8703911, 36)

In [7]:
df_acc.describe()

,accident_year,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,day_of_week,local_authority_district,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag
count,8.703911e+06,8.692273e+06,8.692273e+06,3.816492e+06,3.816492e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703874e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06
mean,1.997127e+03,4.338765e+05,3.004129e+05,-1.404359e+00,5.254946e+01,2.998534e+01,2.786163e+00,1.784516e+00,1.330152e+00,4.151671e+00,3.380266e+02,4.116371e+00,8.213439e+02,5.487847e+00,3.874776e+01,2.492076e+00,1.821230e+00,2.696256e+00,2.087887e+02,6.191125e-03,5.990260e-01,1.998584e+00,1.656446e+00,1.396675e+00,8.176083e-02,8.994164e-02,3.492044e-01,-4.187037e-02,4.979357e-01
std,1.163209e+01,1.060399e+05,1.681665e+05,1.397160e+00,1.429831e+00,2.626146e+01,4.475515e-01,7.272837e-01,8.199040e-01,1.943317e+00,2.577746e+02,1.408903e+00,1.636912e+03,1.817309e+00,1.403551e+01,3.682490e+00,2.363768e+00,3.235886e+00,9.245850e+02,4.227997e-01,1.699612e+00,1.663389e+00,1.725725e+00,6.683733e-01,7.058093e-01,6.254786e-01,1.216595e+00,1.145288e+00,1.472690e+00
min,1.979000e+03,1.000000e+01,1.000000e+01,-7.536169e+00,4.991236e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,1.987000e+03,3.704900e+05,1.776900e+05,-2.317193e+00,5.148732e+01,6.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.020000e+02,3.000000e+00,0.000000e+00,6.000000e+00,3.000000e+01,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
50%,1.996000e+03,4.378500e+05,2.631500e+05,-1.361635e+00,5.222455e+01,2.300000e+01,3.000000e+00,2.000000e+00,1.000000e+00,4.000000e+00,3.060000e+02,4.000000e+00,4.100000e+01,6.000000e+00,3.000000e+01,3.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00
75%,2.006000e+03,5.222200e+05,3.980000e+05,-2.003825e-01,5.345222e+01,4.500000e+01,3.000000e+00,2.000000e+00,1.000000e+00,6.000000e+00,5.170000e+02,6.000000e+00,5.850000e+02,6.000000e+00,5.000000e+01,3.000000e+00,4.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
max,2.021000e+03,9.999800e+05,1.213700e+06,1.762010e+00,6.080166e+01,9.900000e+01,3.000000e+00,1.920000e+02,9.300000e+01,7.000000e+00,9.410000e+02,6.000000e+00,9.999000e+03,1.200000e+01,6.600000e+02,9.900000e+01,9.000000e+00,9.000000e+00,9.999000e+03,9.000000e+00,9.000000e+00,7.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,3.000000e+00,3.000000e+00,2.000000e+00


* overall index col: accident_index
* area key col: lsoa_of_accident_location

### Casualty data

In [11]:
# load data
# all_files = glob.glob(init.raw_path + "/road_safety_data/dft-road-casualty-statistics-casualty-*.csv")

# cas_files = [pd.read_csv(file) for file in all_files] 

# df_cas = pd.concat(cas_files)

# this file has lsoa for all years
df_cas = pd.read_csv(init.raw_path + "/road_safety_data/dft-road-casualty-statistics-casualty-1979-2021.csv")

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,2,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
# check if missings (and if all in same year - file)
df_cas[df_cas.isna().any(axis =1)].groupby('accident_year')['accident_index'].count()

Series([], Name: accident_index, dtype: int64)

In [13]:
df_cas['lsoa_of_casualty'] = df_cas['lsoa_of_casualty'].astype(str)
df_cas['lsoa_of_casualty_missing'] = np.where(df_cas['lsoa_of_casualty'] == '-1', 1, 0)

df_cas[['accident_year','lsoa_of_casualty_missing']].groupby('accident_year').mean()

,lsoa_of_casualty_missing
accident_year,
1979,1.000000
1980,1.000000
1981,1.000000
1982,1.000000
1983,1.000000
1984,1.000000
1985,1.000000
1986,1.000000
1987,1.000000


In [14]:
df_cas.tail()

,accident_index,accident_year,accident_reference,vehicle_reference,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile,lsoa_of_casualty,lsoa_of_casualty_missing
11577516,2021991196607,2021,991196607,1,1,1,1,38,7,3,0,0,0,0,0,9,2,5,-1,1
11577517,2021991196607,2021,991196607,2,2,1,1,61,9,3,0,0,0,0,0,9,2,9,-1,1
11577518,2021991197944,2021,991197944,1,1,2,2,11,3,3,0,0,2,0,0,9,3,5,-1,1
11577519,2021991200639,2021,991200639,1,1,3,2,34,6,3,1,3,0,0,0,0,1,1,-1,1
11577520,2021991201030,2021,991201030,2,1,1,1,58,9,3,0,0,0,0,0,17,3,5,-1,1


* x rows per accidentr_index for each casualty (reference)
* merge on casualty file by accident_index
* ~20% lsoa known for years > 2012 excl 2019: can fill with accident lsoa if missing?

### Vehicle data

In [69]:
# load data
# all_files = glob.glob(init.raw_path + "/road_safety_data/dft-road-casualty-statistics-vehicle-2021.csv")

# veh_files = [pd.read_csv(file) for file in all_files] 

# df_veh = pd.concat(veh_files)

df_veh = pd.read_csv(init.raw_path + "/road_safety_data/dft-road-casualty-statistics-vehicle-1979-2021.csv")

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,2,24,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
# check if missings (and if all in same year - file)
df_veh[df_veh.isna().any(axis =1)].groupby('accident_year')['accident_index'].count()

Series([], Name: accident_index, dtype: int64)

In [71]:
df_veh.head()

,accident_index,accident_year,accident_reference,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,lsoa_of_driver
0,197901A11AD14,1979,01A11AD14,1,109,0,18,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,44,7,-1,-1,-1,-1,-1,-1,-1
1,197901A11AD14,1979,01A11AD14,2,104,0,13,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,197901A1BAW34,1979,01A1BAW34,1,109,0,18,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,197901A1BFD77,1979,01A1BFD77,1,109,0,18,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,21,5,-1,-1,-1,-1,-1,-1,-1
4,197901A1BFD77,1979,01A1BFD77,2,109,0,18,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,44,7,-1,-1,-1,-1,-1,-1,-1


In [79]:
df_veh['lsoa_of_driver'] = df_veh['lsoa_of_driver'].astype(str)
df_veh['lsoa_of_driver_missing'] = np.where(df_veh['lsoa_of_driver'] == '-1', 1, 0)

df_veh[['accident_year','lsoa_of_driver_missing']].groupby('accident_year').mean()

,lsoa_of_driver_missing
accident_year,
1979,1.000000
1980,1.000000
1981,1.000000
1982,1.000000
1983,1.000000
1984,1.000000
1985,1.000000
1986,1.000000
1987,1.000000


* x rows per accidentr_index for each vehicle (reference)
* merge on veh data by accident index
* ~20% lsoa known for years > 2012 excl 2019: can fill with accident lsoa if missing?

## combine data

In [41]:
master_df = pd.merge(left = df_acc.head(20), on = 'accident_index', right = df_veh.head(20), how = 'left')
master_df = pd.merge(left = master_df, on = 'accident_index', right = df_cas.head(20), how = 'left')

In [42]:
master_df.head()

,accident_index,accident_year_x,accident_reference_x,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,accident_year_y,accident_reference_y,vehicle_reference_x,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,accident_year,accident_reference,vehicle_reference_y,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile
0,2016010000005,2016,010000005,519310.0,188730.0,-0.279323,51.584754,1,3,2,1,01/11/2016,3,02:30,28,E09000005,E09000005,3,4006,6,30.0,0,-1,-1,-1,0,0,5,1,1,0,0,1,1,2,E01000543,2016.0,010000005,1.0,8.0,0.0,6.0,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,51.0,8.0,-1.0,-1.0,-1.0,-1.0,3.0,1.0,2016.0,010000005,2.0,1.0,1.0,1.0,23.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0
1,2016010000005,2016,010000005,519310.0,188730.0,-0.279323,51.584754,1,3,2,1,01/11/2016,3,02:30,28,E09000005,E09000005,3,4006,6,30.0,0,-1,-1,-1,0,0,5,1,1,0,0,1,1,2,E01000543,2016.0,010000005,2.0,2.0,0.0,18.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,23.0,5.0,124.0,1.0,4.0,-1.0,4.0,1.0,2016.0,010000005,2.0,1.0,1.0,1.0,23.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0
2,2016010000006,2016,010000006,551920.0,174560.0,0.184928,51.449595,1,3,1,1,01/11/2016,3,00:37,18,E09000004,E09000004,3,207,6,30.0,9,4,6,0,0,0,4,1,1,0,0,1,1,2,E01000375,2016.0,010000006,1.0,9.0,0.0,18.0,5.0,1.0,0.0,2.0,2.0,10.0,1.0,0.0,4.0,1.0,6.0,2.0,36.0,7.0,1461.0,2.0,1.0,-1.0,10.0,1.0,2016.0,010000006,1.0,1.0,1.0,2.0,36.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,10.0
3,2016010000008,2016,010000008,505930.0,183850.0,-0.473837,51.543563,1,3,1,1,01/11/2016,3,01:25,26,E09000017,E09000017,3,4020,1,30.0,1,4,3,4020,0,0,4,1,1,0,0,1,1,2,E01033725,2016.0,010000008,1.0,9.0,0.0,18.0,1.0,8.0,0.0,4.0,5.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,24.0,5.0,1390.0,1.0,5.0,-1.0,8.0,1.0,2016.0,010000008,1.0,1.0,1.0,1.0,24.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,8.0
4,2016010000016,2016,010000016,527770.0,168930.0,-0.164442,51.404958,1,3,1,1,01/11/2016,3,09:15,22,E09000024,E09000024,3,217,6,30.0,3,2,3,217,0,0,1,1,1,0,0,1,1,2,E01003379,2016.0,010000016,1.0,11.0,0.0,18.0,5.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,55.0,8.0,6692.0,2.0,7.0,-1.0,9.0,1.0,2016.0,010000016,1.0,1.0,2.0,2.0,59.0,9.0,3.0,0.0,0.0,0.0,3.0,0.0,11.0,1.0,4.0
5,2016010000018,2016,010000018,510740.0,177230.0,-0.406580,51.483139,1,3,2,1,01/11/2016,3,07:53,25,E09000018,E09000018,3,312,3,40.0,0,-1,-1,-1,0,0,1,1,1,0,0,1,1,2,E01002583,2016.0,010000018,1.0,9.0,0.0,12.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,20.0,4.0,1596.0,1.0,11.0,-1.0,2.0,1.0,2016.0,010000018,2.0,1.0,1.0,1.0,28.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0
6,2016010000018,2016,010000018,510740.0,177230.0,-0.406580,51.483139,1,3,2,1,01/11/2016,3,07:53,25,E09000018,E09000018,3,312,3,40.0,0,-1,-1,-1,0,0,1,1,1,0,0,1,1,2,E01002583,2016.0,010000018,2.0,1.0,0.0,18.0,

* multiple rows per accident if multiple vehicles or casualties.
* should therefore do preprocessing in pipeline and THEN merge together so have:
    * one row per driver (for wfh causation project)
    * one row per accident (for area groups project)

Gives us all accidents from 20x to 2021 with:
* accident info (when, where, why, situation)
* collision victim(s) information if any: who, age, civilian type etc
* vehicle(s) and driver(s) information if any: vehicle situation, type, driver info

* Census data: can be joined on by lsoa: get census data in lsoa granular format.
* Note that NEW LSOA and MSOA codes are created everytime there is a census (as conglomerations/splits of previous LSOA's)
    - so past LSOAs are not overwritten, but will likely not be used in new census
        - Therefore need to get LSOA/MSOA master file for all 2011 and new 2021 LSOAs

In [19]:
df_acc[['police_force',
 'accident_severity',
 'number_of_vehicles',
 'number_of_casualties',
 'date',
 'day_of_week',
 'time',
 'local_authority_district',
 'local_authority_ons_district',
 'local_authority_highway',
 'first_road_class',
 'first_road_number',
 'road_type',
 'speed_limit',
 'junction_detail',
 'junction_control',
 'second_road_class',
 'second_road_number',
 'pedestrian_crossing_human_control',
 'pedestrian_crossing_physical_facilities',
 'light_conditions',
 'weather_conditions',
 'road_surface_conditions',
 'special_conditions_at_site',
 'carriageway_hazards',
 'urban_or_rural_area',
 'did_police_officer_attend_scene_of_accident',
 'trunk_road_flag']].describe()

,police_force,accident_severity,number_of_vehicles,number_of_casualties,day_of_week,local_authority_district,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag
count,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703874e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06,8.703911e+06
mean,2.998534e+01,2.786163e+00,1.784516e+00,1.330152e+00,4.151671e+00,3.380266e+02,4.116371e+00,8.213439e+02,5.487847e+00,3.874776e+01,2.492076e+00,1.821230e+00,2.696256e+00,2.087887e+02,6.191125e-03,5.990260e-01,1.998584e+00,1.656446e+00,1.396675e+00,8.176083e-02,8.994164e-02,3.492044e-01,-4.187037e-02,4.979357e-01
std,2.626146e+01,4.475515e-01,7.272837e-01,8.199040e-01,1.943317e+00,2.577746e+02,1.408903e+00,1.636912e+03,1.817309e+00,1.403551e+01,3.682490e+00,2.363768e+00,3.235886e+00,9.245850e+02,4.227997e-01,1.699612e+00,1.663389e+00,1.725725e+00,6.683733e-01,7.058093e-01,6.254786e-01,1.216595e+00,1.145288e+00,1.472690e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,6.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.020000e+02,3.000000e+00,0.000000e+00,6.000000e+00,3.000000e+01,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
50%,2.300000e+01,3.000000e+00,2.000000e+00,1.000000e+00,4.000000e+00,3.060000e+02,4.000000e+00,4.100000e+01,6.000000e+00,3.000000e+01,3.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00
75%,4.500000e+01,3.000000e+00,2.000000e+00,1.000000e+00,6.000000e+00,5.170000e+02,6.000000e+00,5.850000e+02,6.000000e+00,5.000000e+01,3.000000e+00,4.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
max,9.900000e+01,3.000000e+00,1.920000e+02,9.300000e+01,7.000000e+00,9.410000e+02,6.000000e+00,9.999000e+03,1.200000e+01,6.600000e+02,9.900000e+01,9.000000e+00,9.000000e+00,9.999000e+03,9.000000e+00,9.000000e+00,7.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,3.000000e+00,3.000000e+00,2.000000e+00
